In [ ]:
# Install all the required dependencies for the project
!pip install spacy==2.2.4 --quiet
!python -m spacy download en_core_web_md
!pip install pytorch-lightning==1.6.5 spacy==2.2.4


In [1]:
import numpy as np
import en_core_web_md


In [2]:
nlp = en_core_web_md.load()

In [3]:
# Positive sentences
pos_sentences = ['I love this product',
                 'This is a great movie',
                 'The food was delicious']

# Negative sentences
neg_sentences = ['I hate this product',
                 'This is a terrible movie',
                 'The food was awful']

# Neutral sentences
neu_sentences = ['The weather is nice today',
                 'I am feeling okay',
                 'This book is okay']

# Combine all sentences and create labels
sentences = pos_sentences + neg_sentences + neu_sentences
labels = [1, 1, 1, 0, 0, 0, 2, 2, 2]

In [4]:
# Tokenize and vectorize sentences
max_len = 2
sentences_vectors = []
for sentence in sentences:
  tokens = nlp(sentence)
  vectors = []
  for token in tokens:
    if token.has_vector:
      vectors.append(token.vector)
  vectors = vectors[:max_len] # Truncate vectors if they exceed max length
  vectors += [[0] * 300] * (max_len - len(vectors)) # Pad vectors with zeros
  sentences_vectors.append(vectors)

sentences_vectors = np.array(sentences_vectors)
sentences_vectors

array([[[ 0.18733  ,  0.40595  , -0.51174  , ...,  0.16495  ,
          0.18757  ,  0.53874  ],
        [ 0.13949  ,  0.53453  , -0.25247  , ..., -0.015228 ,
          0.088408 ,  0.30217  ]],

       [[-0.087595 ,  0.35502  ,  0.063868 , ...,  0.03446  ,
         -0.15027  ,  0.40673  ],
        [-0.084961 ,  0.502    ,  0.0023823, ..., -0.21511  ,
         -0.26304  , -0.0060173]],

       [[ 0.27204  , -0.06203  , -0.1884   , ...,  0.13015  ,
         -0.18317  ,  0.1323   ],
        [-0.43512  ,  0.028351 ,  0.4911   , ..., -0.47151  ,
          0.4301   ,  0.36573  ]],

       ...,

       [[ 0.27204  , -0.06203  , -0.1884   , ...,  0.13015  ,
         -0.18317  ,  0.1323   ],
        [ 0.016676 ,  0.33764  ,  0.05375  , ...,  0.14036  ,
         -0.049984 , -0.020776 ]],

       [[ 0.18733  ,  0.40595  , -0.51174  , ...,  0.16495  ,
          0.18757  ,  0.53874  ],
        [-0.08176  ,  0.9505   , -0.19627  , ..., -0.15184  ,
         -0.23864  ,  0.41293  ]],

       [[-0.08759

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

# Split data into training and testing sets
train_vectors = sentences_vectors[:7]
train_labels = labels[:7]
test_vectors = sentences_vectors[7:]
test_labels = labels[7:]

# Define PyTorch dataset
class SentimentDataset(Dataset):
  def __init__(self, vectors, labels):
    self.vectors = torch.tensor(vectors).float()
    self.labels = torch.tensor(labels).long()

  def __getitem__(self, index):
    return self.vectors[index], self.labels[index]

  def __len__(self):
    return len(self.labels)

In [6]:
# Define PyTorch LSTM model
class SentimentLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(SentimentLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, batch):
    h0 = torch.zeros(self.num_layers, batch.size(0), self.hidden_size).to(batch.device)
    c0 = torch.zeros(self.num_layers, batch.size(0), self.hidden_size).to(batch.device)
    # out_lstm - contains the hidden state of the LSTM at each time step
    # I could use it if I need to access all these states.
    # If I need just combined, resulted state I could use "hidden_state" instead.
    out_lstm, (hidden_state, cell_state) = self.lstm(batch, (h0, c0))
    last_lstm_layer = out_lstm[:, -1, :]
    out_linear = self.fc(hidden_state[-1])
    return out_linear

In [7]:
# Define model hyperparameters
input_size = 300
hidden_size = 128
num_layers = 1
num_classes = 3
batch_size = 2

# Create PyTorch data loaders
train_dataset = SentimentDataset(train_vectors, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = SentimentDataset(test_vectors, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create model
model = SentimentLSTM(input_size, hidden_size, num_layers, num_classes)

In [8]:
# Train model
num_epochs = 100
total_step = len(train_loader)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
  for i, (vectors, labels) in enumerate(train_loader):
    outputs = model(vectors)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2 == 0:
      print ('Epoch [{}/{}], Loss: {:.4f}'
             .format(epoch+1, num_epochs, loss.item()))


Epoch [1/100], Loss: 1.0605
Epoch [1/100], Loss: 1.1652
Epoch [2/100], Loss: 1.0560
Epoch [2/100], Loss: 0.8930
Epoch [3/100], Loss: 0.9656
Epoch [3/100], Loss: 0.8717
Epoch [4/100], Loss: 0.8251
Epoch [4/100], Loss: 0.9370
Epoch [5/100], Loss: 0.7312
Epoch [5/100], Loss: 0.8892
Epoch [6/100], Loss: 0.7492
Epoch [6/100], Loss: 0.5670
Epoch [7/100], Loss: 0.8279
Epoch [7/100], Loss: 0.4800
Epoch [8/100], Loss: 0.6321
Epoch [8/100], Loss: 0.4060
Epoch [9/100], Loss: 0.6749
Epoch [9/100], Loss: 0.8454
Epoch [10/100], Loss: 0.4805
Epoch [10/100], Loss: 0.2593
Epoch [11/100], Loss: 0.6484
Epoch [11/100], Loss: 0.1548
Epoch [12/100], Loss: 0.6954
Epoch [12/100], Loss: 0.6798
Epoch [13/100], Loss: 0.3510
Epoch [13/100], Loss: 0.1673
Epoch [14/100], Loss: 0.7279
Epoch [14/100], Loss: 0.1456
Epoch [15/100], Loss: 0.4796
Epoch [15/100], Loss: 0.5937
Epoch [16/100], Loss: 0.0804
Epoch [16/100], Loss: 0.7418
Epoch [17/100], Loss: 0.3810
Epoch [17/100], Loss: 0.5384
Epoch [18/100], Loss: 0.2971
Epo

In [9]:
# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
  for batch in test_loader:
    inputs, labels = batch
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy: {:.2f}%'.format(accuracy))

Accuracy: 0.00%


In [15]:
# test model

# Tokenize the text
text = "nice"

tokens = nlp(text)
vectors = []
for token in tokens:
  if token.has_vector:
    vectors.append(token.vector)
vectors = vectors[:max_len] # Truncate vectors if they exceed max length
vectors += [[0] * 300] * (max_len - len(vectors)) # Pad vectors with zeros

# Convert to PyTorch tensor
inputs = torch.tensor([vectors])

# Pass input through model
model.eval()
with torch.no_grad():
  outputs = model(inputs)

# Get predicted class
probs = torch.softmax(outputs, dim=1)
_, predicted = torch.max(probs.data, 1)

# Print predicted class
print("Predicted class:", predicted)

Predicted class: tensor([1])
